In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import time
from datetime import datetime

class EnhancedExpressionDetector:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.expression_model = None
        self.img_size = 48
        self.class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
        
        # Enhanced colors for different expressions
        self.expression_colors = {
            'angry': (0, 0, 255),        # Red
            'disgust': (0, 128, 0),      # Dark Green
            'fear': (128, 0, 128),       # Purple
            'happy': (0, 255, 255),      # Yellow
            'neutral': (200, 200, 200),  # Light Gray
            'sad': (255, 0, 0),          # Blue
            'surprise': (0, 165, 255)    # Orange
        }
        
        # Expression statistics and tracking
        self.expression_stats = {expr: 0 for expr in self.class_names}
        self.face_tracking = {}  # Track individual faces
        self.next_face_id = 0
        
        # Confidence threshold
        self.confidence_threshold = 0.6
        
        self.load_model()
    
    def load_model(self):
        """Load the trained expression model"""
        model_path = 'models/expression_model.h5'
        if os.path.exists(model_path):
            try:
                self.expression_model = keras.models.load_model(model_path)
                print("✅ Facial expression model loaded successfully!")
                return True
            except Exception as e:
                print(f"❌ Error loading expression model: {e}")
        
        print("❌ No trained expression model found.")
        print("💡 Please train the model first using the main menu")
        return False
    
    def preprocess_face(self, face_img):
        """Enhanced face preprocessing for better accuracy"""
        # Resize to model input size
        face_img = cv2.resize(face_img, (self.img_size, self.img_size))
        
        # Apply histogram equalization for better contrast
        face_img = cv2.equalizeHist(face_img)
        
        # Normalize pixel values
        face_img = face_img.astype('float32') / 255.0
        
        # Reshape for model
        face_img = np.expand_dims(face_img, axis=0)
        face_img = np.expand_dims(face_img, axis=-1)
        
        return face_img
    
    def predict_expression(self, face_img):
        """Predict facial expression with enhanced confidence"""
        if self.expression_model is None:
            return "neutral", 0.5
        
        try:
            # Preprocess face
            processed_face = self.preprocess_face(face_img)
            
            # Predict
            predictions = self.expression_model.predict(processed_face, verbose=0)
            predicted_class = np.argmax(predictions[0])
            confidence = predictions[0][predicted_class]
            
            expression = self.class_names[predicted_class]
            
            # Apply confidence threshold
            if confidence < self.confidence_threshold:
                return "neutral", confidence
            
            return expression, confidence
            
        except Exception as e:
            print(f"❌ Prediction error: {e}")
            return "neutral", 0.0
    
    def assign_face_id(self, face_rect, faces):
        """Assign or track face IDs for consistent tracking"""
        x, y, w, h = face_rect
        center_x = x + w // 2
        center_y = y + h // 2
        
        # Calculate face area for size comparison
        area = w * h
        
        # Check if this face matches any existing tracked face
        min_distance = float('inf')
        best_match_id = None
        
        for face_id, face_data in self.face_tracking.items():
            last_center = face_data['center']
            last_area = face_data['area']
            
            # Calculate distance and size difference
            distance = np.sqrt((center_x - last_center[0])**2 + (center_y - last_center[1])**2)
            area_ratio = min(area, last_area) / max(area, last_area)
            
            # If close enough and similar size, consider it the same face
            if distance < 50 and area_ratio > 0.5:
                if distance < min_distance:
                    min_distance = distance
                    best_match_id = face_id
        
        if best_match_id is not None:
            # Update existing face
            self.face_tracking[best_match_id].update({
                'center': (center_x, center_y),
                'area': area,
                'last_seen': time.time()
            })
            return best_match_id
        else:
            # New face
            face_id = self.next_face_id
            self.face_tracking[face_id] = {
                'center': (center_x, center_y),
                'area': area,
                'last_seen': time.time(),
                'expression_history': []
            }
            self.next_face_id += 1
            return face_id
    
    def cleanup_old_faces(self):
        """Remove faces that haven't been seen for a while"""
        current_time = time.time()
        faces_to_remove = []
        
        for face_id, face_data in self.face_tracking.items():
            if current_time - face_data['last_seen'] > 2.0:  # 2 seconds
                faces_to_remove.append(face_id)
        
        for face_id in faces_to_remove:
            del self.face_tracking[face_id]
    
    def get_stable_expression(self, face_id, current_expression, confidence):
        """Get stable expression using history to reduce flickering"""
        if face_id not in self.face_tracking:
            return current_expression
        
        # Add current expression to history
        self.face_tracking[face_id]['expression_history'].append(
            (current_expression, confidence, time.time())
        )
        
        # Keep only recent history (last 1 second)
        current_time = time.time()
        self.face_tracking[face_id]['expression_history'] = [
            (expr, conf, t) for expr, conf, t in self.face_tracking[face_id]['expression_history']
            if current_time - t < 1.0
        ]
        
        # If we have enough history, use the most frequent expression
        if len(self.face_tracking[face_id]['expression_history']) >= 3:
            expressions = [expr for expr, conf, t in self.face_tracking[face_id]['expression_history']]
            # Count occurrences
            from collections import Counter
            expr_counter = Counter(expressions)
            most_common = expr_counter.most_common(1)[0][0]
            return most_common
        
        return current_expression
    
    def draw_enhanced_face_detection(self, frame, face_rect, expression, confidence, face_id):
        """Draw enhanced face detection with expression info"""
        x, y, w, h = face_rect
        color = self.expression_colors.get(expression, (255, 255, 255))
        
        # Draw main bounding box with thickness based on confidence
        thickness = 2 + int(confidence * 3)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, thickness)
        
        # Draw face ID
        id_text = f"ID: {face_id}"
        cv2.putText(frame, id_text, (x, y - 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # Draw expression label with background
        label_text = f"{expression.upper()}"
        confidence_text = f"Conf: {confidence:.2f}"
        
        # Main label background
        (text_width, text_height), _ = cv2.getTextSize(
            label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2
        )
        
        cv2.rectangle(frame, 
                     (x, y - text_height - 40),
                     (x + text_width + 10, y - 10),
                     color, -1)
        
        # Expression text
        cv2.putText(frame, label_text, (x + 5, y - 25),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Confidence text
        cv2.putText(frame, confidence_text, (x + 5, y - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        # Confidence bar
        bar_width = w
        bar_height = 6
        confidence_width = int(bar_width * confidence)
        
        # Bar background
        cv2.rectangle(frame, 
                     (x, y + h + 5),
                     (x + bar_width, y + h + 5 + bar_height),
                     (50, 50, 50), -1)
        
        # Confidence level
        cv2.rectangle(frame, 
                     (x, y + h + 5),
                     (x + confidence_width, y + h + 5 + bar_height),
                     color, -1)
        
        # Face center point
        center_x = x + w // 2
        center_y = y + h // 2
        cv2.circle(frame, (center_x, center_y), 3, color, -1)
        
        # Draw facial landmarks (approximate)
        self.draw_facial_landmarks(frame, x, y, w, h, expression)
        
        return frame
    
    def draw_facial_landmarks(self, frame, x, y, w, h, expression):
        """Draw approximate facial landmarks based on expression"""
        center_x = x + w // 2
        center_y = y + h // 2
        
        # Eye positions
        left_eye = (center_x - w//4, center_y - h//6)
        right_eye = (center_x + w//4, center_y - h//6)
        
        # Mouth position
        mouth_y = center_y + h//4
        
        # Draw eyes based on expression
        if expression == 'happy':
            # Happy eyes (curved up)
            cv2.ellipse(frame, left_eye, (w//12, h//20), 0, 0, 180, (255, 255, 255), 1)
            cv2.ellipse(frame, right_eye, (w//12, h//20), 0, 0, 180, (255, 255, 255), 1)
            # Smile
            cv2.ellipse(frame, (center_x, mouth_y), (w//4, h//10), 0, 0, 180, (255, 255, 255), 2)
        
        elif expression == 'sad':
            # Sad eyes
            cv2.circle(frame, left_eye, w//20, (255, 255, 255), -1)
            cv2.circle(frame, right_eye, w//20, (255, 255, 255), -1)
            # Frown
            cv2.ellipse(frame, (center_x, mouth_y + h//20), (w//4, h//10), 0, 180, 360, (255, 255, 255), 2)
        
        elif expression == 'surprise':
            # Surprised eyes (larger)
            cv2.circle(frame, left_eye, w//15, (255, 255, 255), -1)
            cv2.circle(frame, right_eye, w//15, (255, 255, 255), -1)
            # Open mouth
            cv2.circle(frame, (center_x, mouth_y), w//10, (255, 255, 255), 2)
        
        elif expression == 'angry':
            # Angry eyes (slanted)
            cv2.line(frame, (left_eye[0]-w//20, left_eye[1]-h//20), 
                    (left_eye[0]+w//20, left_eye[1]+h//20), (255, 255, 255), 2)
            cv2.line(frame, (right_eye[0]-w//20, right_eye[1]-h//20), 
                    (right_eye[0]+w//20, right_eye[1]+h//20), (255, 255, 255), 2)
            # Angry mouth
            cv2.line(frame, (center_x-w//4, mouth_y), (center_x+w//4, mouth_y), (255, 255, 255), 2)
        
        else:  # neutral, disgust, fear
            # Normal eyes and mouth
            cv2.circle(frame, left_eye, w//25, (255, 255, 255), -1)
            cv2.circle(frame, right_eye, w//25, (255, 255, 255), -1)
            cv2.line(frame, (center_x-w//4, mouth_y), (center_x+w//4, mouth_y), (255, 255, 255), 2)
    
    def draw_enhanced_info_panel(self, frame, detected_faces, fps, processing_time):
        """Draw enhanced information panel"""
        panel_width = 350
        panel_height = frame.shape[0]
        
        # Create semi-transparent panel
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (panel_width, panel_height), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.8, frame, 0.2, 0, frame)
        
        y_offset = 30
        line_height = 25
        
        # Title
        cv2.putText(frame, "🎭 MULTI-FACE EXPRESSION DETECTOR", (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        y_offset += line_height * 2
        
        # Performance info
        cv2.putText(frame, f"📊 FPS: {fps:.1f}", (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        y_offset += line_height
        
        cv2.putText(frame, f"⏱️ Processing: {processing_time:.1f}ms", (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        y_offset += line_height
        
        cv2.putText(frame, f"👥 Faces Detected: {len(detected_faces)}", (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        y_offset += line_height
        
        cv2.putText(frame, f"🎯 Confidence: {self.confidence_threshold:.1f}", (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        y_offset += line_height * 2
        
        # Current face detections
        if detected_faces:
            cv2.putText(frame, "CURRENT FACES:", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            y_offset += line_height
            
            for face_data in detected_faces:
                face_id = face_data['face_id']
                expression = face_data['expression']
                confidence = face_data['confidence']
                color = self.expression_colors.get(expression, (255, 255, 255))
                
                face_text = f"Face {face_id}: {expression} ({confidence:.2f})"
                cv2.putText(frame, face_text, (10, y_offset),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                y_offset += line_height
        else:
            cv2.putText(frame, "No faces detected", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 100), 1)
            y_offset += line_height
        
        y_offset += line_height
        
        # Expression statistics
        cv2.putText(frame, "EXPRESSION STATISTICS:", (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
        y_offset += line_height
        
        total_detections = sum(self.expression_stats.values())
        if total_detections > 0:
            sorted_stats = sorted(self.expression_stats.items(), 
                                key=lambda x: x[1], reverse=True)
            
            for expr, count in sorted_stats[:5]:  # Top 5
                if count > 0:
                    percentage = (count / total_detections) * 100
                    color = self.expression_colors.get(expr, (255, 255, 255))
                    stat_text = f"{expr}: {count} ({percentage:.1f}%)"
                    cv2.putText(frame, stat_text, (10, y_offset),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                    y_offset += line_height
        else:
            cv2.putText(frame, "No data yet", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)
            y_offset += line_height
        
        # Color legend
        y_offset = panel_height - 180
        cv2.putText(frame, "EXPRESSION COLORS:", (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        y_offset += line_height
        
        for expr in self.class_names:
            color = self.expression_colors.get(expr, (255, 255, 255))
            cv2.putText(frame, f"● {expr}", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
            y_offset += 15
        
        # Controls help
        y_offset = panel_height - 30
        controls = "Q:Quit  S:Save  +/-:Confidence  C:Clear"
        cv2.putText(frame, controls, (10, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        return frame
    
    def detect_multiple_faces_real_time(self):
        """Enhanced real-time detection for multiple faces"""
        if self.expression_model is None:
            print("❌ Cannot start without a trained model.")
            return
        
        # Initialize camera
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        if not cap.isOpened():
            print("❌ Error: Could not open camera")
            return
        
        print("🚀 Starting Enhanced Multi-Face Expression Detection")
        print("✅ Camera initialized successfully!")
        print("\n🎮 ENHANCED CONTROLS:")
        print("   Q - Quit application")
        print("   S - Save current frame")
        print("   + - Increase confidence threshold")
        print("   - - Decrease confidence threshold")
        print("   C - Clear statistics")
        print("   R - Reset face tracking")
        
        # Performance tracking
        prev_time = time.time()
        fps = 0
        frame_count = 0
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("❌ Failed to grab frame")
                    break
                
                # Calculate FPS
                current_time = time.time()
                fps = 1.0 / (current_time - prev_time)
                prev_time = current_time
                frame_count += 1
                
                # Flip frame horizontally for mirror effect
                frame = cv2.flip(frame, 1)
                
                detected_faces = []
                processing_time = 0
                
                # Convert to grayscale for face detection
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                # Enhanced face detection with timing
                start_time = time.time()
                faces = self.face_cascade.detectMultiScale(
                    gray, 
                    scaleFactor=1.1, 
                    minNeighbors=6, 
                    minSize=(80, 80),
                    flags=cv2.CASCADE_SCALE_IMAGE
                )
                
                # Process each face individually
                for (x, y, w, h) in faces:
                    # Extract face ROI
                    face_roi = gray[y:y+h, x:x+w]
                    
                    # Assign face ID for tracking
                    face_id = self.assign_face_id((x, y, w, h), faces)
                    
                    # Predict expression
                    expression, confidence = self.predict_expression(face_roi)
                    
                    # Get stable expression using history
                    stable_expression = self.get_stable_expression(face_id, expression, confidence)
                    
                    # Update statistics
                    self.expression_stats[stable_expression] += 1
                    
                    # Store detection
                    detected_faces.append({
                        'face_id': face_id,
                        'expression': stable_expression,
                        'confidence': confidence,
                        'bbox': (x, y, w, h)
                    })
                    
                    # Draw enhanced detection for this face
                    frame = self.draw_enhanced_face_detection(
                        frame, (x, y, w, h), stable_expression, confidence, face_id
                    )
                
                processing_time = (time.time() - start_time) * 1000
                
                # Clean up old faces
                self.cleanup_old_faces()
                
                # Draw enhanced information panel
                frame = self.draw_enhanced_info_panel(frame, detected_faces, fps, processing_time)
                
                # Display frame
                cv2.imshow('Enhanced Multi-Face Expression Detection', frame)
                
                # Handle keyboard input
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    # Save current frame
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"multi_face_detection_{timestamp}.jpg"
                    cv2.imwrite(filename, frame)
                    print(f"💾 Frame saved as: {filename}")
                elif key == ord('+'):
                    # Increase confidence threshold
                    self.confidence_threshold = min(0.9, self.confidence_threshold + 0.05)
                    print(f"🔺 Confidence threshold: {self.confidence_threshold:.2f}")
                elif key == ord('-'):
                    # Decrease confidence threshold
                    self.confidence_threshold = max(0.3, self.confidence_threshold - 0.05)
                    print(f"🔻 Confidence threshold: {self.confidence_threshold:.2f}")
                elif key == ord('c'):
                    # Clear statistics
                    self.expression_stats = {expr: 0 for expr in self.class_names}
                    print("📊 Statistics cleared!")
                elif key == ord('r'):
                    # Reset face tracking
                    self.face_tracking = {}
                    self.next_face_id = 0
                    print("🔄 Face tracking reset!")
        
        except KeyboardInterrupt:
            print("\n⏹️ Detection stopped by user")
        
        finally:
            # Print final statistics
            self.print_detailed_stats()
            
            # Cleanup
            cap.release()
            cv2.destroyAllWindows()
            print("✅ Enhanced detection stopped.")
    
    def print_detailed_stats(self):
        """Print detailed detection statistics"""
        print("\n" + "=" * 60)
        print("📊 DETAILED DETECTION STATISTICS")
        print("=" * 60)
        
        total_detections = sum(self.expression_stats.values())
        
        if total_detections > 0:
            print("Expression Breakdown:")
            for expr, count in sorted(self.expression_stats.items(), key=lambda x: x[1], reverse=True):
                if count > 0:
                    percentage = (count / total_detections) * 100
                    print(f"   {expr:10}: {count:4} detections ({percentage:5.1f}%)")
            
            print(f"\n📈 Total detections: {total_detections}")
            print(f"👥 Unique faces tracked: {self.next_face_id}")
        else:
            print("   No expressions detected during this session.")

def main():
    print("🎭 ENHANCED MULTI-FACE EXPRESSION DETECTION")
    print("=" * 60)
    print("Features:")
    print("• Individual face tracking with IDs")
    print("• Stable expression detection (reduces flickering)")
    print("• Enhanced visualization with facial landmarks")
    print("• Real-time statistics and performance metrics")
    print("• Confidence threshold adjustment")
    
    # Initialize detector
    detector = EnhancedExpressionDetector()
    
    # Start enhanced detection
    detector.detect_multiple_faces_real_time()

if __name__ == "__main__":
    main()

🎭 ENHANCED MULTI-FACE EXPRESSION DETECTION
Features:
• Individual face tracking with IDs
• Stable expression detection (reduces flickering)
• Enhanced visualization with facial landmarks
• Real-time statistics and performance metrics
• Confidence threshold adjustment


✅ Facial expression model loaded successfully!
🚀 Starting Enhanced Multi-Face Expression Detection
✅ Camera initialized successfully!

🎮 ENHANCED CONTROLS:
   Q - Quit application
   S - Save current frame
   + - Increase confidence threshold
   - - Decrease confidence threshold
   C - Clear statistics
   R - Reset face tracking
🔻 Confidence threshold: 0.55
🔻 Confidence threshold: 0.50
🔻 Confidence threshold: 0.45
🔻 Confidence threshold: 0.40
🔻 Confidence threshold: 0.35
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 Confidence threshold: 0.30
🔻 